In [1]:
import sys
import importlib

sys.path.append("./src/")
import pdb_numpy

In [2]:
importlib.reload(pdb_numpy)

<module 'pdb_numpy' from './src/pdb_numpy/__init__.py'>

In [3]:
test = pdb_numpy.Coor('./src/pdb_numpy/tests/input/2rri.pdb')

INFO:pdb_numpy.coor:Succeed to read file src/pdb_numpy/tests/input/2rri.pdb 
479 atoms found


In [4]:
test.crystal_pack

'CRYST1    1.000    1.000    1.000  90.00  90.00  90.00 P 1           1          \n'

In [5]:
test.model[0].resname

array([b'HIS', b'HIS', b'HIS', b'HIS', b'HIS', b'HIS', b'HIS', b'HIS',
       b'HIS', b'HIS', b'HIS', b'HIS', b'HIS', b'HIS', b'HIS', b'HIS',
       b'HIS', b'HIS', b'SER', b'SER', b'SER', b'SER', b'SER', b'SER',
       b'SER', b'SER', b'SER', b'SER', b'SER', b'ASP', b'ASP', b'ASP',
       b'ASP', b'ASP', b'ASP', b'ASP', b'ASP', b'ASP', b'ASP', b'ASP',
       b'ASP', b'ALA', b'ALA', b'ALA', b'ALA', b'ALA', b'ALA', b'ALA',
       b'ALA', b'ALA', b'ALA', b'VAL', b'VAL', b'VAL', b'VAL', b'VAL',
       b'VAL', b'VAL', b'VAL', b'VAL', b'VAL', b'VAL', b'VAL', b'VAL',
       b'VAL', b'VAL', b'VAL', b'PHE', b'PHE', b'PHE', b'PHE', b'PHE',
       b'PHE', b'PHE', b'PHE', b'PHE', b'PHE', b'PHE', b'PHE', b'PHE',
       b'PHE', b'PHE', b'PHE', b'PHE', b'PHE', b'PHE', b'PHE', b'THR',
       b'THR', b'THR', b'THR', b'THR', b'THR', b'THR', b'THR', b'THR',
       b'THR', b'THR', b'THR', b'THR', b'THR', b'ASP', b'ASP', b'ASP',
       b'ASP', b'ASP', b'ASP', b'ASP', b'ASP', b'ASP', b'ASP', b'ASP',
      

In [6]:
test = pdb_numpy.Coor('./src/pdb_numpy/tests/input/2rri.pdb')
print(test.len)
assert test.len == 479

test_CA = test.select_atoms("name CA")
print(test_CA.len)
assert test_CA.len == 29

INFO:pdb_numpy.coor:Succeed to read file src/pdb_numpy/tests/input/2rri.pdb 
479 atoms found


479
29


In [7]:
test_CA.model[0].res_num

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [8]:
test_CA.model[0].resname = 'ALA'
test_CA.model[0].resname

array([b'ALA', b'ALA', b'ALA', b'ALA', b'ALA', b'ALA', b'ALA', b'ALA',
       b'ALA', b'ALA', b'ALA', b'ALA', b'ALA', b'ALA', b'ALA', b'ALA',
       b'ALA', b'ALA', b'ALA', b'ALA', b'ALA', b'ALA', b'ALA', b'ALA',
       b'ALA', b'ALA', b'ALA', b'ALA', b'ALA'], dtype='|S4')

In [9]:
import pdb_numpy._alignement as alignement


In [10]:

importlib.reload(alignement)
seq_1 = 'AQDMVSPPPPIADEPLTVNTGIYLIECYSLDDKAETFKVNAFLSLSWKDRRLAFDPV\
RSGVRVKTYEPEAIWIPEIRFVNVENARDADVVDISVSPDGTVQYLERFSARVLSPLDFRRYPFDSQTLHIYLIVRSV\
DTRNIVLAVDLEKVGKNDDVFLTGWDIESFTAVVKPANFALEDRLESKLDYQLRISRQYFSYIPNIILPMLFILFISW\
TAFWSTSYEANVTLVVSTLIAHIAFNILVETNLPKTPYMTYTGAIIFMIYLFYFVAVIEVTVQHYLKVESQPARAASI\
TRASRIAFPVVFLLANIILAFLFFGF'
seq_2 = 'APSEFLDKLMGKVSGYDARIRPNFKGPPVNVTCNIFINSFGSIAETTMDYRVNIFLR\
QQWNDPRLAYSEYPDDSLDLDPSMLDSIWKPDLFFANEKGANFHEVTTDNKLLRISKNGNVLYSIRITLVLACPMDLK\
NFPMDVQTCIMQLESFGYTMNDLIFEWDEKGAVQVADGLTLPQFILKEEKDLRYCTKHYNTGKFTCIEARFHLERQMG\
YYLIQMYIPSLLIVILSWVSFWINMDAAPARVGLGITTVLTMTTQSSGSRASLPKVSYVKAIDIWMAVCLLFVFSALL\
EYAAVNFIARAGTKLFISRAKRIDTVSRVAFPLVFLIFNIFYWITY\
KLVPR'


In [11]:
align_seq_1, align_seq_2 = alignement.align_seq(seq_1, seq_2)

In [12]:
alignement.print_align_seq(align_seq_1, align_seq_2, line_len=80)

--------------AQDMVSPPPPIADEPLTVNTGIYLIECYSLDDKAETFKVNAFLSLSWKDRRLAFDPV-RSGVRVKT
              | * |   * |   *||*| |*|| |  *| | |  ||** **  |*|* ***||    ||| |  
APSEFLDKLMGKVSGYD-ARIRPNFKGPPVNVTCNIFINSFGSIAETTMDYRVNIFLRQQWNDPRLAYSEYPDDSLDLDP

YEPEAIWIPEIRFVNVENARDADV-VD---ISVSPDGTVQYLERFSARVLSPLDFRRYPFDSQTLHIYLIVRSVDTRNIV
   ||** *|| *|* ||*|  |* |*   | |* |*|* *  *||  |  *|*||||*|* **  | *   |   ||||
SMLDSIWKPDLFFANEKGANFHEVTTDNKLLRISKNGNVLYSIRITLVLACPMDLKNFPMDVQTCIMQLESFGYTMNDLI

LAVDLE-KVGKNDDVFLTGWDIESFTAVVKPANFALEDRLESKLDYQLRISRQYFSYIPNIILPMLFILFISWTAFW-S-
|  * |  *   * | *  | |||    || ||   |    | || |||||**   *| || |* *|*|||**||** | 
FEWDEKGAVQVADGLTLPQFILKE-EKDLRYCTKHYNTGKFTCIEARFHLERQMGYYLIQMYIPSLLIVILSWVSFWINM

TSYEANVTLVVSTLIAHIAFNILVETNLPKTPYMTYTGAIIFMIYLFYFVAVIEVTVQHYL-KVESQ--PARAASITRAS
 |  *|* * ||*|||  | |   |||***| *|      | |  ** * *||* || ||| || ||   |**  *  |*
DAAPARVGLGITTVLTMTTQSSGSRASLPKVSYVKAIDIWMAVCLLFVFSALLEYAAVNFIARAGTKLFISRAKRIDTVS

RIAFPVVFLLANIILAFLFFGF--

In [13]:
seq_1 = 'AQDMVSPPPPIADEPLTVNTGIYLIECYSLDDKAETFKVNAFLSLSWKDRRLAFDPV\
RSGVRVKTYEPEAIWIPEIRFVNVENARDADVVDISVSPDGTVQYLERFSARVLSPLDFRRYPFDSQTLHIYLIVRSV\
DTRNIVLAVDLEKVGKNDDVFLTGWDIESFTAVVKPANFALEDRLESKLDYQLRISRQYFSYIPNIILPMLFILFISW\
TAFWSTSYEANVTLVVSTLIAHIAFNILVETNLPKTPYMTYTGAIIFMIYLFYFVAVIEVTVQHYLKVESQPARAASI\
TRASRIAFPVVFLLANIILAFLFFGF'
seq_2 = 'AQDMVSPPPPIADEPLTVNTGIYLIECYSLDDKAETFKVNAFLSLSWKDRRLAFDPV'
align_seq_1, align_seq_2 = alignement.align_seq(seq_1, seq_2)
alignement.print_align_seq(align_seq_1, align_seq_2, line_len=80)

AQDMVSPPPPIADEPLTVNTGIYLIECYSLDDKAETFKVNAFLSLSWKDRRLAFDPVRSGVRVKTYEPEAIWIPEIRFVN
*********************************************************                       
AQDMVSPPPPIADEPLTVNTGIYLIECYSLDDKAETFKVNAFLSLSWKDRRLAFDPV-----------------------

VENARDADVVDISVSPDGTVQYLERFSARVLSPLDFRRYPFDSQTLHIYLIVRSVDTRNIVLAVDLEKVGKNDDVFLTGW
                                                                                
--------------------------------------------------------------------------------

DIESFTAVVKPANFALEDRLESKLDYQLRISRQYFSYIPNIILPMLFILFISWTAFWSTSYEANVTLVVSTLIAHIAFNI
                                                                                
--------------------------------------------------------------------------------

LVETNLPKTPYMTYTGAIIFMIYLFYFVAVIEVTVQHYLKVESQPARAASITRASRIAFPVVFLLANIILAFLFFGF
                                                                             
-----------------------------------------------------------------------------

Identity seq1: 17.98%
Identity se

In [14]:
seq_1 = 'AQDMVSPPPPIADEPLTVNTGIYLIECYSLDDKAETFKVNAFLSLSWKDRRLAFDPV\
RSGVRVKTYEPEAIWIPEIRFVNVENARDADVVDISVSPDGTVQYLERFSARVLSPLDFRRYPFDSQTLHIYLIVRSV\
DTRNIVLAVDLEKVGKNDDVFLTGWDIESFTAVVKPANFALEDRLESKLDYQLRISRQYFSYIPNIILPMLFILFISW\
TAFWSTSYEANVTLVVSTLIAHIAFNILVETNLPKTPYMTYTGAIIFMIYLFYFVAVIEVTVQHYLKVESQPARAASI\
TRASRIAFPVVFLLANIILAFLFFGF'
seq_2 = 'LFYFVAVIEVTVQHYLKVESQPARAASITRASRIAFPVVFLLANIILAFLFFGF'
align_seq_1, align_seq_2 = alignement.align_seq(seq_1, seq_2)
alignement.print_align_seq(align_seq_1, align_seq_2, line_len=80)

AQDMVSPPPPIADEPLTVNTGIYLIECYSLDDKAETFKVNAFLSLSWKDRRLAFDPVRSGVRVKTYEPEAIWIPEIRFVN
                                                                                
--------------------------------------------------------------------------------

VENARDADVVDISVSPDGTVQYLERFSARVLSPLDFRRYPFDSQTLHIYLIVRSVDTRNIVLAVDLEKVGKNDDVFLTGW
                                                                                
--------------------------------------------------------------------------------

DIESFTAVVKPANFALEDRLESKLDYQLRISRQYFSYIPNIILPMLFILFISWTAFWSTSYEANVTLVVSTLIAHIAFNI
                                                                                
--------------------------------------------------------------------------------

LVETNLPKTPYMTYTGAIIFMIYLFYFVAVIEVTVQHYLKVESQPARAASITRASRIAFPVVFLLANIILAFLFFGF
                       ******************************************************
-----------------------LFYFVAVIEVTVQHYLKVESQPARAASITRASRIAFPVVFLLANIILAFLFFGF

Identity seq1: 17.03%
Identity se